In [1]:
import numpy as np
import pandas as pd
import os, pickle, copy
import scipy.io as sio
import platform 
os.chdir('../expt1/task/')
import settings_sustAttnWM
os.chdir('../../scripts/')

# List files

In [2]:
#project directory 
if platform.system()=='Darwin':
    base_project_dir='/Users/megan/Dropbox/writing/articles/2020_pupil/'
else:
    base_project_dir='/Users/Paul Keene/documents/github/pupilometry_paper/'
project_name = 'eyeSustAttnWM01'
project_dir = base_project_dir + 'expt1/'
beh_dir = project_dir + 'beh/'
eye_dir = project_dir + 'eye/'
figure_dir = base_project_dir + 'figures/'
results_dir = project_dir + 'results/'

#index subject names
beh_fn = [f for f in os.listdir(beh_dir ) if f.endswith(project_name + '_explog.csv')]
eye_fn_stim = [f for f in os.listdir(eye_dir ) if f.endswith(project_name + '_ENCARRAY_EYE_SEG.mat')]
eye_fn_ret = [f for f in os.listdir(eye_dir ) if f.endswith(project_name + '_EYE_SEG.mat')]
pick_fn = [f for f in os.listdir(beh_dir ) if f.endswith(project_name + '_expdat.p')]

# Load files and create attention CSV

In [3]:
#initialize arrays
beh_dat = dict.fromkeys(beh_fn)
nsubj = len(beh_fn)

tpts = np.arange(100,900)
nt = np.size(tpts)

for isubj,ifn in enumerate(beh_fn):
    print(isubj,ifn,eye_fn_ret[isubj])

    #load behavioral data from CSV 
    if ifn=='0905181_eyeSustAttnWM01_explog.csv' or ifn=='0917181_eyeSustAttnWM01_explog.csv':
        beh_dat = pd.read_csv(beh_dir+ifn ,header=11)
    else:
        beh_dat = pd.read_csv(beh_dir+ifn ,header=12) 
    itrials = np.arange(np.size(beh_dat.acc))
    
    pick_dat = pickle.load(open(beh_dir+pick_fn[isubj],'rb'),encoding='latin1') 

    #load eye-tracking data
    mat_contents = sio.loadmat(eye_dir+eye_fn_stim[isubj],struct_as_record=False)
    eye_data_stim = mat_contents['eyeData'][0][0]
    s_bl = np.mean(eye_data_stim.trial[0][0].pa[:,0,:100],axis=1)
    s_stim = np.mean(eye_data_stim.trial[0][0].pa[:,0,100:900],axis=1)
    s_stim = np.mean(eye_data_stim.trial[0][0].pa[:,0,100:900],axis=1)
    s_post = np.mean(eye_data_stim.trial[0][0].pa[:,0,100:2501],axis=1)
    s_stim_ptp = np.ptp(eye_data_stim.trial[0][0].pa[:,0,100:900],axis=1)
    s_post_ptp = np.ptp(eye_data_stim.trial[0][0].pa[:,0,100:2501],axis=1)
    
    #delete trials with missing eye data
    if (ifn == '0907182_eyeSustAttnWM01_explog.csv'): 
        j=np.arange(719,721)
    elif (ifn == '0911184_eyeSustAttnWM01_explog.csv'):
        j=np.arange(3094,3104)
    elif (ifn == '0912183_eyeSustAttnWM01_explog.csv'):
        j=np.append(np.arange(256,261),np.arange(1711,1721))
    elif (ifn == '0917182_eyeSustAttnWM01_explog.csv'):
        j=np.arange(204,208)
    else:
        j=[]
           
    arf = ((eye_data_stim.arf[0][0].parserBlinks[0]+eye_data_stim.arf[0][0].missingPupil[0]+eye_data_stim.arf[0][0].saccadeX[0]+eye_data_stim.arf[0][0].saccadeY[0])>0)*1
    
    if np.size(j)>0:
        itrials = np.delete(itrials,j) 
    
    s_trail = np.empty(np.shape(s_stim))
    s_trail[:] = np.nan
    s_trailarf = np.empty(np.shape(s_stim))
    s_trailarf[:] = np.nan
    for ii in range(np.size(itrials)):
        if ii%800>4 & (ii-(itrials[ii-4]))==4:
            s_trail[ii-4] = s_post[ii]
            s_trailarf[ii-4] = arf[ii]
    
    n = np.shape(pick_dat.wholereport_rts)[0]*np.shape(pick_dat.wholereport_rts)[1]
    colorind = np.reshape(pick_dat.trials_colorind,(n,6))
    
    # create attention CSV
    df_attn = pd.DataFrame(columns= ["subject_num","block_num","trial_num","rt_trail","s_bl","s_pretrial","s_ret","wm_perf","triggered_low","triggered_high"])
    i = itrials
    df_attn['subject_num'] = np.tile(isubj,np.size(i))
    df_attn['block_num'] = np.ravel(beh_dat.block[i])
    df_attn['trial_num'] = np.ravel(beh_dat.trial[i])
    df_attn['rt_trail'] = np.ravel(beh_dat.rt_trailingavg[i])
    df_attn['rt_slowthresh'] = np.ravel(beh_dat.rt_runningslowthresh[i])
    df_attn['rt_fastthresh'] = np.ravel(beh_dat.rt_runningfastthresh[i])
    df_attn['rts'] = np.ravel(beh_dat.rts[i])
    df_attn['acc'] = np.ravel(beh_dat.acc[i])
    df_attn['freq_trials'] = np.ravel(beh_dat.freq_trials[i])
    df_attn['s_stim'] = s_stim
    df_attn['s_bl'] = s_bl
    df_attn['s_post'] = s_post
    df_attn['s_trail'] = s_trail
    df_attn['s_trailarf'] = s_trailarf
    df_attn['triggered_low'] = np.ravel(beh_dat.rt_triggered_fast)[i-1]
    df_attn['triggered_high'] = np.ravel(beh_dat.rt_triggered_slow)[i-1]
    df_attn['arf'] = arf
    df_attn['colorind0'] = colorind[i,0]
    df_attn['colorind1'] = colorind[i,1]
    df_attn['colorind2'] = colorind[i,2]
    df_attn['colorind3'] = colorind[i,3]
    df_attn['colorind4'] = colorind[i,4]
    df_attn['colorind5'] = colorind[i,5]

    if isubj==0:
        df = copy.deepcopy(df_attn)
    else:
        df = df.append(df_attn)

df.to_csv(results_dir + 'expt1_attn_results.csv',na_rep="NA")

0 0904181_eyeSustAttnWM01_explog.csv 0904181_eyeSustAttnWM01_EYE_SEG.mat
1 0904182_eyeSustAttnWM01_explog.csv 0904182_eyeSustAttnWM01_EYE_SEG.mat
2 0904183_eyeSustAttnWM01_explog.csv 0904183_eyeSustAttnWM01_EYE_SEG.mat
3 0905181_eyeSustAttnWM01_explog.csv 0905181_eyeSustAttnWM01_EYE_SEG.mat
4 0905183_eyeSustAttnWM01_explog.csv 0905183_eyeSustAttnWM01_EYE_SEG.mat
5 0906181_eyeSustAttnWM01_explog.csv 0906181_eyeSustAttnWM01_EYE_SEG.mat
6 0906182_eyeSustAttnWM01_explog.csv 0906182_eyeSustAttnWM01_EYE_SEG.mat
7 0906183_eyeSustAttnWM01_explog.csv 0906183_eyeSustAttnWM01_EYE_SEG.mat
8 0907181_eyeSustAttnWM01_explog.csv 0907181_eyeSustAttnWM01_EYE_SEG.mat
9 0907182_eyeSustAttnWM01_explog.csv 0907182_eyeSustAttnWM01_EYE_SEG.mat
10 0907183_eyeSustAttnWM01_explog.csv 0907183_eyeSustAttnWM01_EYE_SEG.mat
11 0907184_eyeSustAttnWM01_explog.csv 0907184_eyeSustAttnWM01_EYE_SEG.mat
12 0910181_eyeSustAttnWM01_explog.csv 0910181_eyeSustAttnWM01_EYE_SEG.mat
13 0910182_eyeSustAttnWM01_explog.csv 0910182_ey

# Load files and create WM CSV

In [4]:
beh_dat = dict.fromkeys(beh_fn)
nsubj = len(beh_fn)
tpts = np.arange(0,4501)
nt_ret = np.size(tpts)

tpts_ret = np.arange(2501,4501)
tpts_preret = np.arange(100,2501)
tpts_preret_bl = np.arange(0,100)
tpts_prevstim = np.arange(1700,2501)
tpts_prevstim_bl = np.arange(1600,1700)

for isubj,ifn in enumerate(beh_fn):
    print(isubj,ifn,eye_fn_ret[isubj])

    #load data  
    if ifn=='0905181_eyeSustAttnWM01_explog.csv' or ifn=='0917181_eyeSustAttnWM01_explog.csv':
        beh_dat = pd.read_csv(beh_dir+ifn ,header=11)
    else:
        beh_dat = pd.read_csv(beh_dir+ifn ,header=12) 
    pick_dat = pickle.load(open(beh_dir+pick_fn[isubj],'rb'),encoding='latin1') 
    mat_contents = sio.loadmat(eye_dir+eye_fn_ret[isubj],struct_as_record=False)
    
    #calculate eye data 
    eye_data = mat_contents['eyeData'][0][0]
    s_ret_raw = eye_data.trial[0][0].pa[:,0,tpts]
    s_bl = np.nanmean(s_ret_raw[:,0:100],axis=1)
    s_ret = s_ret_raw-np.tile(s_bl,(nt_ret,1)).T
    
    #calculate behavioral data 
    probe_ind = np.where(np.ravel(beh_dat.probe_trials)==1)[0] #index probes
    
    #deal with files for which eye data was missing
    if (ifn == '0907182_eyeSustAttnWM01_explog.csv'): 
        k=np.arange(719,721)
    elif (ifn == '0911184_eyeSustAttnWM01_explog.csv'):
        k=np.arange(3094,3104)
    elif (ifn == '0912183_eyeSustAttnWM01_explog.csv'):
        k=np.append(np.arange(256,261),np.arange(1711,1721))
    elif (ifn == '0917182_eyeSustAttnWM01_explog.csv'):
        k=np.arange(203,208)
    else:
        k=[]
        
    if np.size(k)>0:
        probe_ind = np.setdiff1d(probe_ind,k)

    n = np.shape(pick_dat.wholereport_rts)[0]*np.shape(pick_dat.wholereport_rts)[1]
    probes_perloc_rts = np.reshape(pick_dat.wholereport_rts,(n,6))[probe_ind]
    probes_perloc_respind = np.reshape(pick_dat.wholereport_respcolorind,(n,6))[probe_ind]
    probes_perloc_colorind = np.reshape(pick_dat.trials_colorind,(n,6))[probe_ind]
    probes_perloc_resporder_raw = np.reshape(pick_dat.wholereport_resporder,(n,6))[probe_ind]
    
    probes_perloc_acc = (probes_perloc_colorind==probes_perloc_respind)*1
    probes_perloc_resporder_new = np.zeros(np.shape(probes_perloc_rts),dtype=int)
    for i in range(np.size(probe_ind)):
        probes_perloc_resporder_new[i] = np.argsort(probes_perloc_rts[i])     
    
    probes_perresporder_acc = np.zeros(np.shape(probes_perloc_rts),dtype=int)
    for i in range(np.size(probe_ind)):
        for j in range(6):
            probes_perresporder_acc[i,j] = probes_perloc_acc[i][probes_perloc_resporder_new[i,j]]
        
    #calculate mean acc 
    probe_perloc_meanacc = np.mean(probes_perloc_acc,axis=0)
    idx = np.argsort(-1*probe_perloc_meanacc)
    
    probes_sortedbylocacc_acc = np.empty(np.shape(probes_perloc_rts),dtype=int)
    for i in range(np.size(probe_ind)):
        for j in range(6):
            probes_sortedbylocacc_acc[i,j] = probes_perloc_acc[i][idx[j]]
            
    df = pd.DataFrame(columns= ["subject_num","block_num","trial_num","rt_trail","s_bl","s_pretrial","s_ret","wm_perf","triggered_low","triggered_high"])
    i = probe_ind
    df['subject_num'] = np.tile(isubj,np.size(i))
    df['block_num'] = np.ravel(beh_dat.block[i])
    df['trial_num'] = np.ravel(beh_dat.trial[i])
    df['rt_trail'] = np.ravel(beh_dat.rt_trailingavg[i-1])
    df['triggered_low'] = np.ravel(beh_dat.rt_triggered_fast)[i-1]
    df['triggered_high'] = np.ravel(beh_dat.rt_triggered_slow)[i-1]
    df['wm_perf'] = np.ravel(beh_dat.wholereportrespacc[i])
    df['wm_perf_subjmean'] = np.ravel(np.mean(beh_dat.wholereportrespacc[i]))*np.ones(np.size(i))
    df['arf'] = eye_data.arf[0][0].parserBlinks[0]+eye_data.arf[0][0].missingPupil[0]+eye_data.arf[0][0].saccadeX[0]+eye_data.arf[0][0].saccadeY[0]
    df['s_ret'] = np.mean(s_ret_raw[:,tpts_ret],axis=1)
    df['s_preret'] = np.mean(s_ret_raw[:,tpts_preret],axis=1)
    df['s_preret_bl'] = np.mean(s_ret_raw[:,tpts_preret_bl],axis=1)
    df['s_prevstim'] = np.mean(s_ret_raw[:,tpts_prevstim],axis=1)
    df['s_prevstim_bl'] = np.mean(s_ret_raw[:,tpts_prevstim_bl],axis=1)
    df['probes_perloc_rts0'] = probes_perloc_rts[:,0]
    df['probes_perloc_rts1'] = probes_perloc_rts[:,1]
    df['probes_perloc_rts2'] = probes_perloc_rts[:,2]
    df['probes_perloc_rts3'] = probes_perloc_rts[:,3]
    df['probes_perloc_rts4'] = probes_perloc_rts[:,4]
    df['probes_perloc_rts5'] = probes_perloc_rts[:,5]
    df['probes_perloc_respind0'] = probes_perloc_respind[:,0]
    df['probes_perloc_respind1'] = probes_perloc_respind[:,1]
    df['probes_perloc_respind2'] = probes_perloc_respind[:,2]
    df['probes_perloc_respind3'] = probes_perloc_respind[:,3]
    df['probes_perloc_respind4'] = probes_perloc_respind[:,4]
    df['probes_perloc_respind5'] = probes_perloc_respind[:,5]
    df['probes_perloc_colorind0'] = probes_perloc_colorind[:,0]
    df['probes_perloc_colorind1'] = probes_perloc_colorind[:,1]
    df['probes_perloc_colorind2'] = probes_perloc_colorind[:,2]
    df['probes_perloc_colorind3'] = probes_perloc_colorind[:,3]
    df['probes_perloc_colorind4'] = probes_perloc_colorind[:,4]
    df['probes_perloc_colorind5'] = probes_perloc_colorind[:,5]
    df['probes_perloc_acc0'] = probes_perloc_acc[:,0]
    df['probes_perloc_acc1'] = probes_perloc_acc[:,1]
    df['probes_perloc_acc2'] = probes_perloc_acc[:,2]
    df['probes_perloc_acc3'] = probes_perloc_acc[:,3]
    df['probes_perloc_acc4'] = probes_perloc_acc[:,4]
    df['probes_perloc_acc5'] = probes_perloc_acc[:,5]
    df['probes_perloc_resporder0'] = probes_perloc_resporder_new[:,0]
    df['probes_perloc_resporder1'] = probes_perloc_resporder_new[:,1]
    df['probes_perloc_resporder2'] = probes_perloc_resporder_new[:,2]
    df['probes_perloc_resporder3'] = probes_perloc_resporder_new[:,3]
    df['probes_perloc_resporder4'] = probes_perloc_resporder_new[:,4]
    df['probes_perloc_resporder5'] = probes_perloc_resporder_new[:,5]
    df['probes_perresporder_acc0'] = probes_perresporder_acc[:,0]
    df['probes_perresporder_acc1'] = probes_perresporder_acc[:,1]
    df['probes_perresporder_acc2'] = probes_perresporder_acc[:,2]
    df['probes_perresporder_acc3'] = probes_perresporder_acc[:,3]
    df['probes_perresporder_acc4'] = probes_perresporder_acc[:,4]
    df['probes_perresporder_acc5'] = probes_perresporder_acc[:,5]
    df['probes_sortedbylocacc_acc0'] = probes_sortedbylocacc_acc[:,0]
    df['probes_sortedbylocacc_acc1'] = probes_sortedbylocacc_acc[:,1]
    df['probes_sortedbylocacc_acc2'] = probes_sortedbylocacc_acc[:,2]
    df['probes_sortedbylocacc_acc3'] = probes_sortedbylocacc_acc[:,3]
    df['probes_sortedbylocacc_acc4'] = probes_sortedbylocacc_acc[:,4]
    df['probes_sortedbylocacc_acc5'] = probes_sortedbylocacc_acc[:,5]

    if isubj==0:
        df2 = copy.deepcopy(df)
    else:
        df2 = df2.append(df)

df2.to_csv(results_dir + 'expt1_wm_results.csv',na_rep="NA")


0 0904181_eyeSustAttnWM01_explog.csv 0904181_eyeSustAttnWM01_EYE_SEG.mat


/Users/megan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Mean of empty slice


1 0904182_eyeSustAttnWM01_explog.csv 0904182_eyeSustAttnWM01_EYE_SEG.mat
2 0904183_eyeSustAttnWM01_explog.csv 0904183_eyeSustAttnWM01_EYE_SEG.mat
3 0905181_eyeSustAttnWM01_explog.csv 0905181_eyeSustAttnWM01_EYE_SEG.mat
4 0905183_eyeSustAttnWM01_explog.csv 0905183_eyeSustAttnWM01_EYE_SEG.mat
5 0906181_eyeSustAttnWM01_explog.csv 0906181_eyeSustAttnWM01_EYE_SEG.mat
6 0906182_eyeSustAttnWM01_explog.csv 0906182_eyeSustAttnWM01_EYE_SEG.mat
7 0906183_eyeSustAttnWM01_explog.csv 0906183_eyeSustAttnWM01_EYE_SEG.mat
8 0907181_eyeSustAttnWM01_explog.csv 0907181_eyeSustAttnWM01_EYE_SEG.mat
9 0907182_eyeSustAttnWM01_explog.csv 0907182_eyeSustAttnWM01_EYE_SEG.mat
10 0907183_eyeSustAttnWM01_explog.csv 0907183_eyeSustAttnWM01_EYE_SEG.mat
11 0907184_eyeSustAttnWM01_explog.csv 0907184_eyeSustAttnWM01_EYE_SEG.mat
12 0910181_eyeSustAttnWM01_explog.csv 0910181_eyeSustAttnWM01_EYE_SEG.mat
13 0910182_eyeSustAttnWM01_explog.csv 0910182_eyeSustAttnWM01_EYE_SEG.mat
14 0910183_eyeSustAttnWM01_explog.csv 0910183_e